In [ ]:
from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer
from datetime import date, timezone
from dateutil.relativedelta import relativedelta

import datetime
import sqlite3
import pandas as pd
import re

In [ ]:
gis = GIS('Pro')

# Define the earliest date considered as active
OLDEST_ACTIVE_DATE = (date.today() - relativedelta(years=1)).isoformat()

In [ ]:
# Connect to the SQLite database
connection = sqlite3.connect("agol_management.db")

# Define the SQL query
sql_query = f"""
SELECT u.Owner AS User
FROM (
    SELECT Owner
    FROM items
    GROUP BY Owner
    HAVING COUNT(*) = COUNT(CASE WHEN Share_level = 'private' THEN 1 END)
) AS u
LEFT JOIN users d ON d.user = u.Owner
WHERE d.last_enabled < '{OLDEST_ACTIVE_DATE}' AND d.lastlogin < '{OLDEST_ACTIVE_DATE}'
"""

# Execute the SQL query and retrieve long inactive users with only private content
liu_opc = pd.read_sql_query(sql_query, connection)

# Close the database connection
connection.close()

In [ ]:
# Read the Organization Items AGOL report
org_items = pd.read_csv('agol-logs/OrganizationItems_latest.csv')

# Filter out df 
liu_items_df = org_items[org_items['Owner'] == 'null0007_UMN'] # This is a liu example
liu_items = []

# Create list of item objects
for i in range(len(liu_items_df)):
    item = gis.content.get(liu_items_df['Item ID'].iloc[i])
    liu_items.append(item)

### Optional:

Run this cell if you want to get the item details in a csv file 

In [ ]:
# Initialize a list to store item information
items_info = []

# Iterate over search results
for item in liu_items:

    # Convert UNIX timestamp to date string
    last_viewed_date = datetime.datetime.fromtimestamp(item.lastViewed / 1000, timezone.utc).strftime('%Y-%m-%d')
    last_modified_date = datetime.datetime.fromtimestamp(item.modified / 1000, timezone.utc).strftime('%Y-%m-%d')
    
    # Get item properties
    item_info = {
        'Item ID': item.id,
        'Title': item.title,
        'Owner': item.owner,
        'Type': item.type,
        'Size': item.size/(1024**3), # Convert to GB
        'Last Modified': last_modified_date,
        'Number of Views': item.numViews,
        'Last Viewed': last_viewed_date,
        'Can delete' : item.can_delete
    }
    # Append item information to the list
    items_info.append(item_info)

# Convert the list of dictionaries to a DataFrame
content_df = pd.DataFrame(items_info)
content_df.to_csv("items.csv", index=False, float_format="%f")

### Feature layers that have views

In [ ]:
def feature_service_views(item):
    """
    Retrieves the IDs of views associated with a Feature Service item.

    Args:
    - item: The item to check. Must be of type 'Feature Service'.

    Returns:
    - List of view IDs associated with the Feature Service.

    Raises:
    - ValueError: If the item is not of type 'Feature Service'.
    """
    view_ids = []

    # Check if the item is a Feature Service
    if item.type == "Feature Service":
        
        # Get the list of views associated with the Feature Service
        view_list = item.view_manager.list()
        
        # If there are views associated with the Feature Service
        if len(view_list) > 0:
            
            # Iterate through each view and add the ID of the view to view_ids list
            for view in view_list:
                view_ids.append(view.id) 
    else:
        # Raise an error if the item is not a Feature Service
        raise ValueError("Item is not a Feature Service")
    
    return view_ids

In [ ]:
def layer_and_views_access_status(layer):
    """
    Checks the access status of a layer and its associated views.

    Args:
    - layer: A feature layer object.

    Returns:
    - status: A boolean indicating whether the layer and its views are fully private.
    """

    # Initialize status as False
    status = False

    # Get a list of views associated with the layer
    view_list = layer.view_manager.list()

    # Check if the layer itself is private
    if layer.access == 'private':
        status = True

    # If there are views associated with the layer
    if len(view_list) > 0:
        view_list_status = []

        # Check access status of each view
        for view in view_list:
            view_list_status.append(view.access)

        # If any view is not private, set status to False
        if not all(i == 'private' for i in view_list_status):
            status = False

    return status

In [ ]:
def source_and_dependents(item):
    """
    Retrieves the source and dependents of an item.

    Args:
    - item: An ArcGIS item object.

    Returns:
    - out_dict: A dictionary containing information about the source and dependents.
    """

    # Define the types of relations between items
    relation_types = ['Map2Service',
                    'WMA2Code',
                    'Map2FeatureCollection',
                    'MobileApp2Code',
                    'Service2Data',
                    'Service2Service',
                    'Map2AppConfig',
                    'Item2Attachment',
                    'Item2Report',
                    'Listed2Provisioned',
                    'Style2Style',
                    'Service2Style',
                    'Survey2Service',
                    'Survey2Data',
                    'Service2Route',
                    'Area2Package',
                    'Map2Area',
                    'Service2Layer',
                    'Area2CustomPackage',
                    'TrackView2Map',
                    'SurveyAddIn2Data',
                    'WorkforceMap2FeatureService',
                    'Theme2Story',
                    'WebStyle2DesktopStyle',
                    'Solution2Item',
                    'APIKey2Item',
                    'Mission2Item',
                    'Notebook2WebTool']

    # Get all sources of the layer
    sources = item.related_items(rel_type='Service2Data')
    
    # Initialize lists to store information about all sources and dependents
    source_ids = []
    all_dependents = []
    
    # Iterate through each source
    for source in sources:
        # Get the ID of the source and add it to the list
        source_ids.append(source.id)

        # Iterate through each relation type
        for rel_type in relation_types:
            try:                        
                # Get the dependents of the source
                dependents = source.related_items(rel_type=rel_type, direction='reverse')
        
                # Add dependents to the list of all dependents
                all_dependents.extend(dependents)

            except KeyError:
                # If the relation type doesn't exist, ignore and continue
                pass
    
    # Remove duplicates from the list of dependents
    all_dependents = list(set(all_dependents))
    
    # Remove the current item ID from the list of dependents
    other_dependents = [dependent.id for dependent in all_dependents if dependent.id != item.id]
    
    # Create a dictionary to store the information
    out_dict = {'Item': item.id, 'Sources': source_ids, 'Other dependents': other_dependents}
    
    return out_dict

In [ ]:
# Print info for Feature Services that are not views
for item in liu_items:
    if item.type == "Feature Service":
        try:
            view_list = item.view_manager.list()
            print(f"{source_and_dependents(item)}, Fully private: {layer_and_views_access_status(item)}, # Views: {len(view_list)}")
        except AttributeError:
            print(f'Error with item {item.id}')

In [ ]:
private_size = 0

for item in liu_items:
    try:
        print(source_and_dependents(item))
        # Calculate the total size of the private content
        if item.access == 'private':
            private_size += item.size/(1024**3) # Convert to GB
    except AttributeError:
        print(f"Error with item {item.id}")

print("Private content size (GB): ", round(private_size, 2))

### Web Maps

In [ ]:
def get_all_layers(layer, layers_list):
    """
    Recursively retrieves all individual layers from a given layer, including nested group layers.

    Args:    
    - layer: The current layer (which may be an individual layer or a group layer).
    - layers_list: The list to which individual layers are added.
    """
    if 'layerType' in layer and layer['layerType'] != 'GroupLayer':
        layers_list.append(layer)
    elif 'layers' in layer:
        for sub_layer in layer['layers']:
            get_all_layers(sub_layer, layers_list)

def list_all_layers_in_webmap(webmap):
    """
    Lists all individual layers in a given web map item.
    
    Args:
    - webmap: The web map object from which to list all layers.
    
    Returns: 
    - wm_layers: A list of all individual layers in the web map.
    """
    wm_layers = []
    for layer in webmap.layers:
        get_all_layers(layer, wm_layers)
    return wm_layers

In [ ]:
def find_url_in_layer(layer):
    """
    Finds the URL in the layer dictionary by searching for keys that contain 'url' (case-insensitive).
    
    Args:
    - layer: The layer dictionary.

    Returns: 
    - The URL if found, None otherwise.
    """
    url_pattern = re.compile(r'url', re.IGNORECASE)
    
    for key, value in layer.items():
        if url_pattern.search(key):
            return value
    
    return None

In [ ]:
def webmap_access_status(wm_item, urls, org_items):
    """
    Determine the access status of a web map item based on its layers' access levels.

    Args:
    - wm_item (object): An ArcGIS item object corresponding to a web map.
    - urls (list): A list of URLs of the layers in the web map.
    - org_items (DataFrame): A DataFrame containing organizational items with 'Item Url' and 'Share Level' columns.

    Returns:
    - bool: True if the web map or all its layers are private, False otherwise.
    """

    private = False  # Initialize private flag
    wm_ly_access = []  # List to store access levels of layers

    if wm_item.access == 'private':
        private = True  # Set private flag if web map is private
    else:
        for url in urls:
            try:
                # Find the layer's access status in the Organization Items report
                ly_url = url.rsplit('/', 1)[-2]
                ly_item = org_items[org_items['Item Url'] == ly_url]
                ly_access = ly_item['Share Level'].iloc[0]
                wm_ly_access.append(ly_access)
            except:
                pass  # Ignore errors and continue

        # Check if all layers are private
        if len(wm_ly_access) == len(urls) and all(i == 'private' for i in wm_ly_access):
            private = True

    return private

##### Flag empty Web Maps

In [ ]:
# Read the Basemap URLs file
basemap_urls = pd.read_excel('basemaps_URLs.xlsx')

basemap_ids = []
# Remove confusing strings from URLs and extract basemap IDs
for i in range(len(basemap_urls)):
    basemap_urls['URLs'].iloc[i] = basemap_urls['URLs'].iloc[i].replace('https://', '')
    basemap_urls['URLs'].iloc[i] = basemap_urls['URLs'].iloc[i].replace('cdn.', '')
    parts = basemap_urls['URLs'].iloc[i].split('/')
    for part in parts:
        if len(part) == 32:
            basemap_ids.append(part)
            break

In [ ]:
def check_urls_in_dict(unique_urls, nested_dict):
    """
    Check if any of the unique URLs are found in the given nested dictionary.

    Args:
    - unique_urls (list): A list of unique URLs.
    - nested_dict (dict): The nested dictionary to search for URLs.

    Returns:
    - bool: True if any URL is found, False otherwise.
    """
    # Convert nested_dict to string for easier URL checking
    nested_str = str(nested_dict)

    # Check if any of the unique URLs exist in the string representation of nested_dict
    for url in unique_urls:
        if url in nested_str:
            return True

    # If no URLs are found in the nested dictionary, return False
    return False

In [ ]:
empty_wm = []
private_wms =[]

# Iterate over each web map in the content list
for item in liu_items:
    # Skip the item if it is not of type 'Web Map'
    if item.type != 'Web Map':
        continue

    # Create a WebMap object from the item
    wm = WebMap(item)

    # Initialize lists for webmap's layers info
    wm_layers = []
    wm_urls =[]
    bm_layers = 0

    # Check if the web map has no layers and no tables, marking it as empty
    if len(wm.layers) == 0 and len(wm.tables) == 0 and check_urls_in_dict(basemap_urls['URLs'], wm.basemap):
        empty_wm.append(item.id)
    else:
        # Get all layers in the web map, including nested ones
        wm_layers = list_all_layers_in_webmap(wm)
        deleted_layers = 0

        # Iterate over each layer in the web map
        for layer in wm_layers:
            # Find the URL of the layer
            url = find_url_in_layer(layer)
            wm_urls.append(url)

            if url is None:
                # If the layer does not have a URL, it is a sketch
                pass
            else:
                fl = FeatureLayer(url)
                try:
                    # Try to access the properties of the FeatureLayer
                    fl.properties
                except:
                    # Increment the deleted_layers counter if the layer properties cannot be accessed
                    deleted_layers += 1

            # Check if the layer is a basemaps
            if check_urls_in_dict(basemap_urls['URLs'], layer):
                bm_layers += 1
            else:
                for bm_id in basemap_ids:
                    if bm_id in str(layer):
                        bm_layers += 1

        # If all layers are deleted (inaccessible) and there are no tables, mark the web map as empty
        if deleted_layers == len(wm_layers) and len(wm.tables) == 0:
            empty_wm.append(item.id)

        if bm_layers == len(wm_layers) and len(wm.tables) == 0:
            empty_wm.append(item.id)

        # Check if the web map is private
        if webmap_access_status(item, wm_urls, org_items):
            private_wms.append(item.id)

# Output the list of empty and private web maps
print("Empty web maps:", empty_wm)
print("Private Web Maps:", private_wms)

### Findings

1. Some Feature Services created from a FGDB or shapefile have two sources: the FGDB and a Service Definition

2. Trying to publish a Service Definition that has already been published yields an error message. Same for uploaded files from which Feature Services were derived. So far, I have found that only Views have sources with more dependents, i.e., other Views. This makes sense as multiple Views can come from the same Feature Service (source).

3. Deleting a private layer from a public web map gives more information to the end user than keeping the layer as a message that reads "Unamble to add layer <layer's name>" is displayed. In contrast, when the layer is kept, the user doens't know what private layer is there.

4. Basemap's styleURL changes when added as a layer

5. When you want to add a basemap as a layer and search for it by item ID, several results are returned from Esri and other sources that correspond to such basemap, but in many cases neither of them have the ID that you inputted in the search bar.